In [1]:
!pip install gymnasium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 6.0 MB/s eta 0:00:00


In [2]:
import gymnasium as gym
import numpy as np
import time
from IPython import display

# FROZEN LAKE V1

In [3]:
env1 = gym.make('FrozenLake-v1', render_mode="ansi")

In [4]:
env1.P[0][3] # Transition model

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.P to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.P` for environment variables or `env.get_wrapper_attr('P')` that will search the reminding wrappers.
  logger.warn(


[(0.3333333333333333, 1, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False)]

In [5]:
env1.observation_space.n

16

In [6]:
env1.action_space.n

4

In [7]:
def play(env, policy, render=False):
    state, _ = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info, _ = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            print(env.render())
            time.sleep(0.5)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [8]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

In [9]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value

        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break

    return v_values

In [10]:
def value_iteration(env, max_iters=500, gamma=0.9):
    # initialize
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []

            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])

                q_values.append(q_value)

            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]

        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break

    return v_values

In [11]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=np.int32)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])

            q_values.append(q_value)

        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action

    return policy

In [12]:
def policy_improvement(env, v_values, gamma = 0.9):
  """
  Performs policy improvement based on the given value function.

  Args:
    env (gym.Env): The OpenAI Gym environment.
    v_values (numpy.ndarray): The value function for each state.
    gamma (float, optional): The discount factor. Defaults to 0.9.

  Returns:
    numpy.ndarray: The improved policy.

  """

  # Initialize the policy with zeros
  policy = np.zeros(env.observation_space.n)

  # Iterate over each state
  for state in range(env.observation_space.n):

    # Initialize the Q-values for each action
    q_values = np.zeros(env.action_space.n)

    # Iterate over each action
    for action in range(env.action_space.n):

      # Iterate over each possible transition from the current state and action
      for prob, next_state, reward, done in env.P[state][action]:

        # Calculate the Q-value for the current state-action pair
        q_values[action] += prob * (reward + gamma * v_values[next_state])

    # Update the policy with the action that maximizes the Q-value
    policy[state] = np.argmax(q_values)

  return policy

In [13]:
# The function policy_iteration is defined with three parameters: env (the environment), max_iters (the maximum number of iterations), and gamma (the discount factor).
def policy_iteration(env, max_iters = 1000, gamma = 0.9):

  # Initialize the policy with a zero array of the same size as the observation space in the environment.
  policy = np.zeros(env.observation_space.n)

  # Start a loop that will run for a maximum of max_iters iterations.
  for i in range(max_iters):

    # Evaluate the current policy to get the value function.
    value_function = policy_evaluation(env,policy)

    # Extract a new policy based on the current value function.
    new_policy = policy_extraction(env,value_function)

    # If the new policy is the same as the old policy (i.e., the policy has converged), break the loop.
    if (np.all(policy == new_policy)):
      break

    # If the policy has not converged, update the policy with the new policy and continue the loop.
    policy = new_policy

  # Return the final policy after the loop has finished.
  return policy

In [14]:
start_time = time.time()
optimal_vvalue1 = value_iteration(env1)
optimal_policy1 = policy_extraction(env1,optimal_vvalue1)
play_multiple_times(env1,optimal_policy1,1000)
end_time = time.time()
print("Value Iteration Running Time:",end_time - start_time)

Converged at 79-th iteration.
Number of successes: 775/1000
Average number of steps: 43.681290322580644
Value Iteration Running Time: 1.004274606704712


In [15]:
start_time = time.time()
optimal_policy1 = policy_iteration(env1)
play_multiple_times(env1,optimal_policy1,1000)
end_time = time.time()
print("Policy Iteration Running Time:",end_time - start_time)

Converged at 0-th iteration.
Converged at 23-th iteration.
Converged at 59-th iteration.
Converged at 62-th iteration.
Converged at 79-th iteration.
Converged at 80-th iteration.
Number of successes: 778/1000
Average number of steps: 43.03341902313625
Policy Iteration Running Time: 0.9025859832763672


In [16]:
optimal_policy1

array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0], dtype=int32)

# FrozenLake8x8-v1

In [17]:
env2 = gym.make('FrozenLake8x8-v1')

In [18]:
env2.observation_space.n

64

In [19]:
env2.action_space.n

4

In [20]:
start_time = time.time()
optimal_vvalue2 = value_iteration(env2)
optimal_policy2 = policy_extraction(env2,optimal_vvalue2)
play_multiple_times(env2,optimal_policy2,1000)
end_time = time.time()
print("Value Iteration Running Time:",end_time - start_time)

Converged at 117-th iteration.
Number of successes: 720/1000
Average number of steps: 72.60277777777777
Value Iteration Running Time: 2.11513352394104


In [21]:
start_time = time.time()
optimal_policy2 = policy_iteration(env2)
play_multiple_times(env2,optimal_policy2,1000)
end_time = time.time()
print("Policy Iteration Running Time:",end_time - start_time)

Converged at 27-th iteration.
Converged at 91-th iteration.
Converged at 92-th iteration.
Converged at 86-th iteration.
Converged at 90-th iteration.
Converged at 92-th iteration.
Converged at 95-th iteration.
Converged at 100-th iteration.
Converged at 112-th iteration.
Converged at 117-th iteration.
Number of successes: 740/1000
Average number of steps: 74.40810810810811
Policy Iteration Running Time: 2.208096504211426


In [22]:
optimal_policy2

array([3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 1, 3, 3, 0, 0, 2, 3,
       2, 1, 3, 3, 3, 1, 0, 0, 2, 1, 3, 3, 0, 0, 2, 1, 3, 2, 0, 0, 0, 1,
       3, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 2, 0, 1, 0, 0, 1, 1, 1, 0],
      dtype=int32)

# Taxi-v3

In [23]:
env3 = gym.make('Taxi-v3')

In [24]:
env3.observation_space.n

500

In [25]:
env3.action_space.n

6

In [26]:
start_time = time.time()
optimal_vvalue3 = value_iteration(env3)
optimal_policy3 = policy_extraction(env3,optimal_vvalue3)
play_multiple_times(env3,optimal_policy3,1000)
end_time = time.time()
print("Value Iteration Running Time:",end_time - start_time)

Converged at 116-th iteration.
Number of successes: 1000/1000
Average number of steps: 13.301
Value Iteration Running Time: 7.952168703079224


In [27]:
start_time = time.time()
optimal_policy3 = policy_iteration(env3)
play_multiple_times(env3,optimal_policy3,1000)
end_time = time.time()
print("Policy Iteration Running Time:",end_time - start_time)

Converged at 88-th iteration.
Converged at 97-th iteration.
Converged at 100-th iteration.
Converged at 101-th iteration.
Converged at 102-th iteration.
Converged at 103-th iteration.
Converged at 106-th iteration.
Converged at 109-th iteration.
Converged at 110-th iteration.
Converged at 111-th iteration.
Converged at 112-th iteration.
Converged at 115-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Number of successes: 1000/1000
Average number of steps: 13.007
Policy Iteration Running Time: 12.71245265007019


In [28]:
optimal_policy3

array([4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 3, 3,
       3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0,
       2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2,
       2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 1, 2, 0, 2,
       1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 2, 1, 2, 3, 2, 3, 3,
       3, 3, 1, 1, 1, 1, 3, 3, 3, 3, 2, 2, 2, 2, 3, 1, 3, 2, 3, 3, 3, 3,
       1, 1, 1, 1, 3, 3, 3, 3, 0, 0, 0, 0, 3, 1, 3, 0, 3, 3, 3, 3, 1, 1,
       1, 1, 3, 3, 3, 3, 0, 0, 0, 0, 3, 1, 3, 0, 1,

# Nhận xét và kết quả

- Policy Iteration và Value Iteration đều cho ra tỉ lệ thành công tương đương nhau (ở Taxi-V3 tỉ lệ thành công là 100%).
- Khi so về thời gian chạy, Value Iteration chạy nhanh hơn Policy Iteration một chút. Đặc biệt ở map 'Taxi-v3', khi độ chênh lệch ở đây là khoảng 5 giây.
- Khi so về trung bình số bước đi, sự chênh lệch của 2 thuật toán là không đánh kể, chỉ chênh nhau một vài bước.
- Nhìn chung, với kết quả và tỉ lệ thành công tương tự nhau sau 1000 lần chạy, Value Iteration vượt Policy Iteration về mặt thời gian. Tuy nhiên sự chênh lệch chỉ thực sự đáng kể khi được thử nghiệm ở map 'Taxi v3'.